In [28]:
import numpy as np
import random
import pandas as pd
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from matplotlib import dates
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime
from lmfit import minimize, Parameters, Parameter, report_fit
from statsmodels.tsa.arima_model import ARIMA

# step 0: data importation #

In [29]:
csv_url="https://raw.githubusercontent.com/ADelau/proj0016-epidemic-data/main/data.csv"
data = pd.read_csv(csv_url)

In [210]:
data.head(5)

,Day,num_positive,num_tested,num_hospitalised,num_cumulative_hospitalizations,num_critical,num_fatalities
0,1,0,0,1,1,0,0
1,2,5,8,1,1,0,0
2,3,10,12,2,2,0,0
3,4,11,16,2,2,0,0
4,5,9,12,2,2,0,0


# step 1: setting up the model #

## step 1.1: writing the ode system ##

In [83]:
def deriv(y, t, N, ps):
    S,E,I,C,R,H,ICU,D = y
    try:
        beta_SE = ps['beta_SE'].value
        gamma_ER = ps['gamma_ER'].value
        fraction_symptomatic = ps['fraction_symptomatic'].value
        days_EtoI=ps['days_EtoI'].value
        days_ItoH=ps['days_ItoH'].value
        days_ItoR=ps['days_ItoR'].value
        days_HtoR=ps['days_HtoR'].value
        days_HtoICU=ps['days_HtoICU'].value
        days_ICUtoD=ps['days_ICUtoD'].value
        days_ICUtoH=ps['days_ICUtoH'].value
        
    except:
        beta_SE, gamma_ER, fraction_symptomatic, days_EtoI, days_ItoH, days_ItoR, days_HtoR, days_HtoICU, days_ICUtoD, days_ICUtoH= ps
    
    #beta = beta_i*(1.1-tau*t) idée de faire régresser beta
    dSdt = -beta_SE*(S*(I+C))/(N-D)
    dEdt = beta_SE*(S*(I+C))/(N-D)-gamma_ER*E
    dIdt = (fraction_symptomatic)*E/days_EtoI-I*(1/days_ItoH+1/days_ItoR)
    dCdt = (1-fraction_symptomatic)*E/days_EtoI-C/days_ItoR
    dRdt = (C+I)/days_ItoR+H/days_HtoR
    dHdt = I/days_ItoH*I+ICU/days_ICUtoH-H*(1/days_HtoICU+1/days_HtoR)
    dICUdt = H/days_HtoICU-ICU*(1/days_ICUtoD+1/days_ICUtoH)
    dDdt = ICU/days_ICUtoD
    
    return dSdt, dEdt, dIdt, dCdt, dRdt, dHdt, dICUdt, dDdt

## step 1.2: writing the parameters values and guesses ##


In [245]:
# the lmfit module uises an orderd dict structure
# to store the parameters to be optimized
# https://lmfit.github.io/lmfit-py/parameters.html

params = Parameters()
params.add('beta_SE', value=0.15, min=0.001, max=2)
params.add('gamma_ER', value= 0.05, min=0.1, max=2)
params.add('fraction_symptomatic', value= 0.6, min=0.5, max=0.7)
params.add('days_EtoI', value= 3, min=1, max=5)
params.add('days_ItoH',value=5, min=1,max=20)
params.add('days_ItoR',value=10,min=1,max=50)
params.add('days_HtoR',value=12,min=1,max=100)
params.add('days_HtoICU',value=5,min=1,max=50)
params.add('days_ICUtoD',value=2,min=1,max=50)
params.add('days_ICUtoH',value=6,min=1,max=50)
params.add('I0',value=5,min=1,max=50)
params.add('E0',value=10,min=1,max=40)
params.add('I0',value=5,min=1,max=20)
params.add('C0',value=5,min=0,max=20)
params.add('OBS_Tr_EI_to_nbTest',value=0.25,min=0.05,max=1)
params.add('OBS_nbTest_to_nbpos',value=0.75,min=0.5,max=0.9)
# params.add('OBS_H_to_ICU',value=0.1,min=0.01,max=0.5) mistake??



## step 1.3 writing the solver ##

In [246]:
#this function solves the ode
#input: function deriv, initial compartiments y0, t, and N,ps as arguments??
#output: the ode solution
def odesol(y,t,N,ps):
    E0=ps['E0'].value
    I0 = ps['I0'].value
    C0=ps['C0'].value
    S0=N-I0-E0-C0-1
    R0=0
    H0=1
    ICU0=0
    D0=0
    
    y0 = S0,E0,I0,C0,R0,H0,ICU0,D0
    x = odeint(deriv, y0, t, args=(N, ps))
    
    return x



In [247]:
# this function generate the observation dataframe
def create_obsdf(model,ps,data):
    result=pd.DataFrame(columns=data.columns)
    result['Day']=data['Day']
    nb_trans_EI=ps['beta_SE']*(model['S']*(model['I']+model['C']))/(10**6-model['D'])
    result['num_tested']=ps['OBS_Tr_EI_to_nbTest'].value*nb_trans_EI
    result['num_positive']=ps['OBS_nbTest_to_nbpos'].value*result['num_tested']
    result['num_hospitalised']=model['H']
    result['num_cumulative_hospitalizations']=result['num_hospitalised'].diff().fillna(0).cumsum()
    result['num_critical']=model['ICU']
    result['num_fatalities']=model['D']
    return result
    

#test['num_tested']=params['OBS_Tr_EI_to_nbTest'].value*

# step 2: fitting the model #

## step 2.1: write objective function ##

In [248]:
obs_df

NameError: name 'obs_df' is not defined

In [249]:
#this function compute residuals,
#ie objective function to be minimized in the optimization function
def residual(ps, ts, data):
    model = pd.DataFrame(odesol(y0,t,N,ps), columns=['S','E','I','C','R','H','ICU','D'])
    obs_df=create_obsdf(model,ps,data)
    component1=(obs_df['num_positive']-data['num_positive']).ravel()
    component2=(obs_df['num_hospitalised']-data['num_hospitalised']).ravel()
    component3=(obs_df['num_critical']-data['num_critical']).ravel()
    component4=(obs_df['num_fatalities']-data['num_fatalities']).ravel()
    # penalty function to think more about !!!
    return component1**2+component2**2+component3**2+component4**2

In [250]:
t = np.linspace(0, data.shape[0]-1, data.shape[0])
residual(params,t,data)

array([7.91012303e+00, 2.47372836e+01, 1.09293917e+02, 2.17510289e+02,
       3.44372083e+02, 5.69120954e+02, 8.84869471e+02, 1.37849149e+03,
       1.90015773e+03, 2.63782862e+03, 4.41197249e+03, 5.98386346e+03,
       8.60434920e+03, 1.25178542e+04, 1.80140752e+04, 2.52930154e+04,
       3.58894987e+04, 5.21876909e+04, 7.42013008e+04, 1.04272335e+05,
       1.47516825e+05, 2.07209847e+05, 2.94302085e+05, 4.17660603e+05,
       5.91771122e+05, 8.37515943e+05, 1.18778008e+06, 1.67986514e+06,
       2.37560620e+06, 3.35474586e+06, 4.73882026e+06, 6.66547172e+06,
       9.41048748e+06, 1.33107147e+07, 1.87408768e+07, 2.63969731e+07,
       3.71750905e+07, 5.24867309e+07, 7.39699454e+07, 1.04420263e+08,
       1.47279916e+08, 2.07471449e+08, 2.92626760e+08, 4.12705649e+08,
       5.81829252e+08, 8.19557546e+08, 1.15518563e+09, 1.62740322e+09,
       2.29283800e+09, 3.23159445e+09, 4.55433579e+09, 6.42043714e+09,
       9.05284398e+09, 1.27696664e+10, 1.80115454e+10, 2.54152625e+10,
      

## step 2.2: write optimization command ##

In [251]:
result = minimize(residual, params, args=(t, data), method='leastsq')

In [252]:
result

In [256]:
final = data + result.residual.reshape(data.shape)
# display fitted statistics
report_fit(result)

ValueError: cannot reshape array of size 71 into shape (71,7)